In [1]:
from llama_index.llms.openai import OpenAI
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

In [2]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

 ········


In [3]:
response = OpenAI().complete("What is Capital of India?")
print(response)

The capital of India is New Delhi.


## Document Loading

In [4]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("D:\Data\LLM\llamaindex\IndianEconomy").load_data()

In [5]:
len(documents)

9

In [6]:
documents[0].metadata

{'page_label': '1',
 'file_name': 'Indian_Economy_Overview.pdf',
 'file_path': 'D:\\Data\\LLM\\llamaindex\\IndianEconomy\\Indian_Economy_Overview.pdf',
 'file_type': 'application/pdf',
 'file_size': 7918,
 'creation_date': '2024-06-09',
 'last_modified_date': '2024-06-09'}

In [7]:
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.0)

In [25]:
#pip install llama_index

In [9]:
from pinecone import Pinecone, ServerlessSpec

In [10]:
import os
from pinecone import Pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
api_key = os.environ.get('80d261ee-5381-4e8f-b3c3-e30a86f5d39d') or '80d261ee-5381-4e8f-b3c3-e30a86f5d39d'

# configure client
pc = Pinecone(api_key=api_key)

os.environ['PINECONE_API_KEY'] = '80d261ee-5381-4e8f-b3c3-e30a86f5d39d'

In [20]:
from pinecone import ServerlessSpec

cloud = os.environ.get('PINECONE_CLOUD') or 'aws'
region = os.environ.get('PINECONE_REGION') or 'us-east-1'

spec = ServerlessSpec(cloud=cloud, region=region)

In [21]:
index_name = 'gen-qa-openai-fast'

In [22]:
# check if index already exists (it shouldn't if this is first time)
if index_name not in pc.list_indexes().names():
    # if does not exist, create index
    pc.create_index(
        index_name,
        dimension=1536,  # dimensionality of text-embedding-ada-002
        metric='cosine',
        spec=spec
    )
# connect to index
index = pc.Index(index_name)
# view index stats
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [23]:
from llama_index.llms.openai import OpenAI

In [43]:
#%pip install llama-index-vector-stores-pinecone

In [24]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.pinecone import PineconeVectorStore
from llama_index.core import StorageContext

In [25]:
vector_store = PineconeVectorStore(pinecone_index=index)
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)

Upserted vectors:   0%|          | 0/9 [00:00<?, ?it/s]

In [26]:
query_engine = index.as_query_engine()
response = query_engine.query("What are the Key Sectors of the Indian Economy?")
print(response)

The key sectors of the Indian economy are agriculture, industry, and services.


In [27]:
response = query_engine.query("What is the Capital of India?")
print(response)

New Delhi


In [29]:
response = query_engine.query("What are the Challenges Facing the Indian Economy?")
print(response)

Unemployment, income inequality, agricultural distress, infrastructure deficit, and regulatory hurdles are the challenges facing the Indian economy.
